In [101]:
import pandas as pd
import numpy as np
import nltk



!pip install demoji
import demoji
demoji.download_codes()

nltk.download('all')


... OK (Got response in 0.18 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data..

True

In [102]:
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import WhitespaceTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.models import *
from keras.layers import *
from keras.callbacks import *
import pandas as pd
import pickle
import numpy as np
import random
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from gensim.models import Word2Vec

In [103]:
col_list = ['Text', 'Label']
csvFile = '/content/drive/MyDrive/IIITD/IR/Project/Mid Sem/Data/Working_copy_22_MAR_Ver2.csv'
csvFile = '/content/drive/MyDrive/IIITD/IR/Project/Mid Sem/Data/TRAIN_DATASET.csv'
data =  pd.read_csv(csvFile, usecols=col_list)
data.Text

0       This actually broke my heart... An elderly Kis...
1       I am a daughter of farmers, of course Iâ€™m go...
2       â€œthe power of people is stronger than the pe...
3       Coming together is the beginning. Keeping toge...
4       The farmers are more articulate and aware than...
                              ...                        
1243    AAP MLA @JarnailSinghAAP arrested for protesti...
1244    This is a brutal and merciless attack by Modi ...
1245    From the barbed wires\n From the showers of wa...
1246    Orders from above - use 'misguised' for #Farme...
1247    AIKS is at the border with the Farmers. The Al...
Name: Text, Length: 1248, dtype: object

In [104]:
data = data[data.Label!='PROVOKING']

In [105]:
data.Label.unique()

array(['PF_AG', 'NEUTRAL', 'PG_AF'], dtype=object)

In [106]:
#======================================================================================================
                       
                                       #PreProcessing

#==============================================================================================
def replaceElongated(word):
    """ Replaces an elongated word with its basic form, unless the word exists in the lexicon(In Most Cases Working) """

    repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
    repl = r'\1\2\3'
    if wordnet.synsets(word):
        return word
    repl_word = repeat_regexp.sub(repl, word)
    if repl_word != word:      
        return replaceElongated(repl_word)
    else:       
        return repl_word

def doPreProcess(text):

   # ----------------------[1.    Demoji]----------------------------------------------
  text=demoji.replace_with_desc(text, sep="")# Repalce with text


  #------------------------[2.   # and @ handling here]--------------------------------
  #Eiether-------------------------------------------
    # text=re.sub("([@#][A-Za-z0-9]+)","",text)#Replace by space those words with # and @

      #Or
  text=re.sub("([@#])","",text)#Replace by space the chars as  # and @
  #-----------------------------------------------------------


  #------------------------[3.    Remove the Links]--------------------------------
  text=re.sub(r"http\S+", "", text)#Remove the Links
 

  #-------------------------[4.   Emoticons Replacement with text]-------------------
  

  tokenizer = RegexpTokenizer(r'\w+')
  words=WhitespaceTokenizer().tokenize(text.lower())#tokenizer.tokenize(text)#Tokenize and casings
  
  #df = pd.read_excel('Emoticons_Emojis_Text.xlsx',sheet_name='Sheet2',header=0,converters={'Emoji':str,'Text':str,'Emoticons':str})
  #dictOfEmoticons=df.set_index('Emoji')['Text'].to_dict()
  #words=[dictOfEmoticons.get(x) if x in dictOfEmoticons.keys() else x for x in words]
  text=' '.join(words)



  #----------------------[5.       Ignore Non Ascii Characters]-----------------------
  encoded_string = text.encode("ascii", "ignore")  #For ASCII Onlys
  text = encoded_string.decode()

  #----------------------[6.       Single Quotes Handling Here Removal]---------------------:
  text=text.translate(str.maketrans("","", "'"))
  # text=text.translate(str.maketrans("","", string.punctuation))


  #---------------------[7.         Tokenizes]----------------------
  # tokenizer = RegexpTokenizer(r'\w+')
  words=WhitespaceTokenizer().tokenize(text)
 
  #---------------------[8.         Lower Case Folding]------------------------------
  # words=tokenizer.tokenize(text.lower())#Tokenize and casings

  #---------------------[9.         Replace Elongated Words]-------------------------------

  words=[replaceElongated(x) if not wordnet.synsets(x) else x for x in  words]





   #---------------------[10. & 11.        Stopwords Removal & Lematize]--------------------------------------------- 
  wnl = WordNetLemmatizer() 
  words=[wnl.lemmatize(word) for word in words]
  words=[word for word in words if word not in stopwords.words('english') ]



  #---------------------[10. & 11.        Stopwords Removal & Lematize]--------------------------------------------- 
  # wnl = WordNetLemmatizer() 
  # words=[wnl.lemmatize(word) for word in words]
  # words=[word for word in words if word not in stopwords.words('english') ]


  

  #finalText= ' '.join(all_words)

  #-------------------[11.                 Pnctations Removal]------------------------------
  #return re.sub(r'[^\w\s]', '',words)
  
  #return words
  return ' '.join(words)

def preProcessing(df):
  
  df=df[['Text']]['Text'].apply(doPreProcess)
  return df

#======================================================================================================
                       
                                       #Global Data Frames

#==============================================================================================

In [107]:
data['Preprocessed_tweets']=data.Text.apply(doPreProcess)

In [108]:
from keras import models
from keras import layers

df = data
df.columns = ['Text', 'Label', 'Tweets']

In [109]:
indexDictionary = {}
index = 0
for cell in df.Tweets:
  for word in cell.split(' '):
    if not (word in indexDictionary):
      indexDictionary[word] = index
      index+=1

In [110]:
#String to list seperated by space
finalList = []
for cell in data.Tweets:
  cellList = []
  for word in cell.split(' '):
    cellList.append(indexDictionary[word])
  finalList.append(cellList)
  

In [111]:
#Numpy array of word indexes
ndata = np.array(finalList)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [112]:
#Replace ANTI_GOVT, ANTI_FARMERS... with 0, 1, 2...
uniqueNames = data.Label.unique()
for i in range(len(uniqueNames)):
  df.Label = df.Label.replace(uniqueNames[i], i)

In [113]:
#Numpy array of targets
targets = df.Label.to_numpy()

In [114]:
dimensions = 8000
def vectorizeData(sequences, dimension = dimensions):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results
    
#Vectorize data
data = vectorizeData(ndata)
targets = np.array(targets).astype("float32")

In [115]:
#Divide into train test with random
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(data, targets, test_size=0.33, random_state = 4)


In [116]:
#Divide into train test with random
from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(data, targets, test_size=0.33, random_state = 4)


model = models.Sequential()
model.add(layers.Dense(50, activation = "relu", input_shape=(8000, )))

model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))

model.add(layers.Dense(1, activation = "sigmoid"))

model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)


results = model.fit(
    xTrain, yTrain,
    epochs= 2,
    batch_size = 500,
    validation_data = (xTest, yTest)
)


print(np.mean(results.history["accuracy"]))
model.summary()

Epoch 1/2
2/2 [==============================] - 1s 434ms/step - loss: 0.0000e+00 - accuracy: 0.5326 - val_loss: 0.0000e+00 - val_accuracy: 0.5909
Epoch 2/2
2/2 [==============================] - 0s 161ms/step - loss: 0.0000e+00 - accuracy: 0.5448 - val_loss: 0.0000e+00 - val_accuracy: 0.5909
0.5410447716712952
Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 50)                400050    
_________________________________________________________________
dropout_29 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_59 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_30 (Dropout)         (None, 50)                0         
_________________________________________________________________
dens

In [147]:
import pickle
def saveData(pic, filename):
  with open(filename, 'wb') as file:
      pickle.dump(pic, file)

pathName = '/content/drive/MyDrive/IIITD/IR/Project/Mid Sem/Data/'

saveData(data, pathName+'numpyData.sav')
saveData(targets, pathName+'numpyTargets.sav')

model.save(pathName + 'model.sav')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/IIITD/IR/Project/Mid Sem/Data/model.sav/assets


In [148]:
def loadData(filename):
  with open(filename, 'rb') as file:
      return pickle.load(file)

data = loadData(pathName+'numpyData.sav')
targets = loadData(pathName+'numpyTargets.sav')
model = keras.models.load_model(pathName + 'model.sav')

In [149]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(data, targets, test_size=0.33, random_state = 4)

results = model.fit(
    xTrain, yTrain,
    epochs= 2,
    batch_size = 500,
    validation_data = (xTest, yTest)
)


print(np.mean(results.history["accuracy"]))
model.summary()

Epoch 1/2
2/2 [==============================] - 1s 302ms/step - loss: 0.0000e+00 - accuracy: 0.5473 - val_loss: 0.0000e+00 - val_accuracy: 0.5909
Epoch 2/2
2/2 [==============================] - 0s 98ms/step - loss: 0.0000e+00 - accuracy: 0.5473 - val_loss: 0.0000e+00 - val_accuracy: 0.5909
0.5472636818885803
Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 50)                400050    
_________________________________________________________________
dropout_29 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_59 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_30 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense

In [150]:
savedModelLink = 'https://drive.google.com/drive/folders/1fH2x6vKV1tXLPCdAjLpGC5pi8UesE__9?usp=sharing'